In [1]:
from ib_insync import *
from get_portfolio import *

In [ ]:
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7496, clientId=14, readonly=True)

Error 10275, reqId -1: Positions info is not available for account(s): U18414148,U18580926 until the application is finished and approved.


<IB connected to 127.0.0.1:7496 clientId=14>

Error 200, reqId 14: No security definition has been found for the request, contract: Contract(conId=759935494, exchange='SMART')
Error 200, reqId 15: No security definition has been found for the request, contract: Contract(conId=759935494, exchange='SMART')
Error 1100, reqId -1: Connectivity between IBKR and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IBKR and Trader Workstation has been lost.
Peer closed connection.


In [3]:
positions = get_current_portfolio()

Request Headers: {'User-Agent': 'Python/3.12.8', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}
Response Headers: {'Referrer-Policy': 'Origin-when-cross-origin', 'Content-Encoding': 'gzip', 'Content-Type': 'text/xml', 'X-Content-Type-Options': 'nosniff', 'Content-Security-Policy': "frame-ancestors 'self'  *.interactivebrokers.com  *.interactivebrokers.ca  *.interactivebrokers.com.hk  *.interactivebrokers.hk  *.interactivebrokers.ch  *.interactivebrokers.eu  *.interactivebrokers.ie  *.interactivebrokers.lu  *.interactivebrokers.hu  *.interactivebrokers.com.sg  *.ibkr.com.sg  *.interactivebrokers.ch  *.interactivebrokers.co.uk  *.interactivebrokers.com.au  *.interactivebrokers.co.jp  *.interactivebrokers.co.in  *.ibkram.com  IBKR.docebosaas.com  *.interactiveadvisors.com  *.ibkr.com  *.ibkr.com.cn  *.clientam.com  *.youtube.com  *.clientam.ch  *.clientam.com.hk  *.go-mpulse.net  *.akstat.io  *.lynxbroker.com  impact.interactivebrokers.com  widgets.tipran

c:\Users\alber\OneDrive - CAAN Alternative Asset Management, S.A\Escritorio\exposition_tracker\get_portfolio.py:151: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  current_positions = pd.concat([positions,trades_today])
c:\Users\alber\OneDrive - CAAN Alternative Asset Management, S.A\Escritorio\exposition_tracker\get_portfolio.py:151: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  current_positions = pd.concat([positions,trades_today])


In [4]:
positions['Expiry'] = pd.to_datetime(positions['Expiry'])
positions['Expiry'] = positions['Expiry'].dt.strftime('%Y%m%d')

In [5]:
options = positions[(positions.AssetClass == "OPT")|(positions.AssetClass == "FOP")]
options

,Description,Symbol,CurrencyPrimary,AssetClass,Quantity,Multiplier,UnderlyingSymbol,ListingExchange,Strike,Expiry,Conid
3,ES 17APR25 5480 P,E3DJ5 P5480,USD,FOP,-3,50,ESM5,CME,5480.0,20250417,769615430
4,ES 17APR25 5670 P,E3DJ5 P5670,USD,FOP,3,50,ESM5,CME,5670.0,20250417,769614950
5,GLNG 17APR25 34 P,GLNG 250417P00034000,USD,OPT,-8,100,GLNG,CBOE,34.0,20250417,762938880
12,SOI 17APR25 57 P,LF1 APR25 57 P,EUR,OPT,-5,100,SOI,MONEP,57.0,20250417,759935494
13,VET 20JUN25 5 P,VET 250620P00005000,USD,OPT,24,100,VET,CBOE,5.0,20250620,736192429


In [6]:
# contract = Contract(conId=769615430, exchange='CME')
# ib.qualifyContracts(contract)

# ticker = ib.reqMktData(contract, '', False, False)
# ib.sleep(2)

# print(contract)
# print("Bid/Ask:", ticker.bid, "/", ticker.ask)
# print("Greeks:", ticker.modelGreeks)

In [7]:
deltas = []

for idx, row in options.iterrows():
    try:
        # Detectar si es una opción sobre futuro (como ES)
        if row['UnderlyingSymbol'].startswith('ES'):
            exchange = 'CME'
        else:
            exchange = 'SMART'

        contract = Contract(conId=int(row['Conid']), exchange=exchange)
        ib.qualifyContracts(contract)

        ticker = ib.reqMktData(contract, '', False, False)
        ib.sleep(2)

        delta = ticker.modelGreeks.delta if ticker.modelGreeks else None

        # Debug si delta no llega
        if delta is None:
            print(f"⚠️ Sin delta para {row['Description']}")
            print(f"  Bid: {ticker.bid}, Ask: {ticker.ask}, Last: {ticker.last}")
            print(f"  Greeks: {ticker.modelGreeks}")

        deltas.append(delta)

    except Exception as e:
        print(f"❌ Error con {row['Description']}: {e}")
        deltas.append(None)

options['Delta'] = deltas
print(options[['Description', 'Delta']])


Unknown contract: Contract(conId=759935494, exchange='SMART')


⚠️ Sin delta para SOI 17APR25 57 P
  Bid: nan, Ask: nan, Last: nan
  Greeks: None
          Description     Delta
3   ES 17APR25 5480 P -0.570089
4   ES 17APR25 5670 P -0.968791
5   GLNG 17APR25 34 P -0.143647
12   SOI 17APR25 57 P       NaN
13    VET 20JUN25 5 P -0.178525


C:\Users\alber\AppData\Local\Temp\ipykernel_39688\1443328488.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  options['Delta'] = deltas


In [8]:
options

,Description,Symbol,CurrencyPrimary,AssetClass,Quantity,Multiplier,UnderlyingSymbol,ListingExchange,Strike,Expiry,Conid,Delta
3,ES 17APR25 5480 P,E3DJ5 P5480,USD,FOP,-3,50,ESM5,CME,5480.0,20250417,769615430,-0.570089
4,ES 17APR25 5670 P,E3DJ5 P5670,USD,FOP,3,50,ESM5,CME,5670.0,20250417,769614950,-0.968791
5,GLNG 17APR25 34 P,GLNG 250417P00034000,USD,OPT,-8,100,GLNG,CBOE,34.0,20250417,762938880,-0.143647
12,SOI 17APR25 57 P,LF1 APR25 57 P,EUR,OPT,-5,100,SOI,MONEP,57.0,20250417,759935494,NaN
13,VET 20JUN25 5 P,VET 250620P00005000,USD,OPT,24,100,VET,CBOE,5.0,20250620,736192429,-0.178525


In [17]:
Contract(conId=736192429)

Contract(conId=736192429)

In [18]:
stock = Stock('AAPL', 'SMART', 'USD')
ib.qualifyContracts(stock)
data = ib.reqMktData(stock)
data

Ticker(contract=Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'))

In [19]:
stockValue = data.marketPrice()
print(stockValue)

200.93


In [20]:
nvda = Stock("NVDA", "SMART", "USD")
ib.qualifyContracts(nvda)
[ticker] = ib.reqTickers(nvda)
nvdaValue = ticker.marketPrice()
nvdaValue

110.46

In [7]:
for row in options.itertuples():
    print(row.UnderlyingSymbol)
    stock = Stock(row.UnderlyingSymbol, "SMART", row.CurrencyPrimary)
    ib.qualifyContracts(stock)
    [ticker] = ib.reqTickers(stock)
    nvdaValue = ticker.marketPrice()
    print(nvdaValue)

Unknown contract: Stock(symbol='ESM5', exchange='SMART', currency='USD')
Unknown contract: Stock(symbol='ESM5', exchange='SMART', currency='USD')


ESM5
nan
ESM5
nan
GLNG
nan
HEIA
nan
SOI
nan
VET
nan
YPF
nan


In [ ]:
nvda = Stock("NVDA", "SMART", "USD")

In [28]:
chains = ib.reqSecDefOptParams(nvda.symbol, "", nvda.secType, nvda.conId)
chain = next(c for c in chains if c.tradingClass == "NVDA" and c.exchange == "SMART")

In [29]:
chain

OptionChain(exchange='SMART', underlyingConId='4815747', tradingClass='NVDA', multiplier='100', expirations=['20250404', '20250411', '20250417', '20250425', '20250502', '20250509', '20250516', '20250620', '20250718', '20250815', '20250919', '20251017', '20251121', '20251219', '20260116', '20260320', '20260618', '20260918', '20261218', '20270115', '20270617', '20271217'], strikes=[0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5, 10.0, 10.5, 11.0, 11.5, 12.0, 12.5, 13.0, 13.5, 14.0, 14.5, 15.0, 15.5, 16.0, 16.5, 17.0, 17.5, 18.0, 18.5, 19.0, 19.5, 20.0, 20.5, 21.0, 21.5, 22.0, 22.5, 23.0, 23.5, 24.0, 24.5, 25.0, 25.5, 26.0, 26.5, 27.0, 27.5, 28.0, 28.5, 29.0, 29.5, 30.0, 30.5, 31.0, 31.5, 32.0, 32.5, 33.0, 33.5, 34.0, 34.5, 35.0, 35.5, 36.0, 36.5, 37.0, 37.5, 38.0, 38.5, 39.0, 39.5, 40.0, 40.5, 41.0, 41.5, 42.0, 42.5, 43.0, 43.5, 44.0, 44.5, 45.0, 45.5, 46.0, 46.5, 47.0, 47.5, 48.0, 48.5, 49.0, 49.5, 50.0, 50.5, 51.0, 51.5, 52.0, 52.5, 53.0, 

In [33]:
strikes = [
    strike for strike in chain.strikes
    if strike % 5 == 0
    and nvdaValue - 20 < strike < nvdaValue + 20
]

expirations = sorted(exp for exp in chain.expirations)[:8]
rights = ["C", "P"]

In [ ]:
contracts = [
    Option("NVDA", expiration, strike, right, "SMART", tradingClass="NVDA")
    for right in rights
    for expiration in expirations
    for strike in strikes
]

In [35]:
contracts

[Option(conId=761446201, symbol='NVDA', lastTradeDateOrContractMonth='20250404', strike=90.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='NVDA  250404C00090000', tradingClass='NVDA'),
 Option(conId=761446252, symbol='NVDA', lastTradeDateOrContractMonth='20250404', strike=95.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='NVDA  250404C00095000', tradingClass='NVDA'),
 Option(conId=761444654, symbol='NVDA', lastTradeDateOrContractMonth='20250404', strike=100.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='NVDA  250404C00100000', tradingClass='NVDA'),
 Option(conId=761444714, symbol='NVDA', lastTradeDateOrContractMonth='20250404', strike=105.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='NVDA  250404C00105000', tradingClass='NVDA'),
 Option(conId=761444750, symbol='NVDA', lastTradeDateOrContractMonth='20250404', strike=110.0, right='C', multiplier='100', exchange='

In [ ]:
contracts = ib.qualifyContracts(*contracts)
tickers = ib.reqTickers(*contracts)

contractData = [
    (
        t.contract.lastTradeDateOrContractMonth, 
        t.contract.strike, 
        t.contract.right,
        t.time,
        t.close,
        nvdaValue,
    )
    for t
    in tickers
]

fields = [
    "expiration", 
    "strike", 
    "right", 
    "time", 
    "undPrice",
    "close", 
]

util.df([dict(zip(fields, t)) for t in contractData])

,expiration,strike,right,time,undPrice,close
0,20250404,90.0,C,2025-04-02 11:29:10.789489+00:00,20.23,108.35
1,20250404,95.0,C,2025-04-02 11:29:11.151127+00:00,15.27,108.35
2,20250404,100.0,C,2025-04-02 11:29:11.152332+00:00,10.42,108.35
3,20250404,105.0,C,2025-04-02 11:29:10.793003+00:00,5.98,108.35
4,20250404,110.0,C,2025-04-02 11:29:10.793131+00:00,2.55,108.35
5,20250404,115.0,C,2025-04-02 11:29:10.793204+00:00,0.63,108.35
6,20250404,120.0,C,2025-04-02 11:29:11.151474+00:00,0.09,108.35
7,20250404,125.0,C,2025-04-02 11:29:11.153297+00:00,0.02,108.35
8,20250411,90.0,C,2025-04-02 11:29:11.151191+00:00,20.44,108.35
9,20250411,95.0,C,2025-04-02 11:29:11.151960+00:00,15.62,108.35


In [ ]:
ib.disconnect()

TypeError: IB.disconnect() got an unexpected keyword argument 'clientId'